# Imports

In [1]:
from sys import path as syspath
syspath.append("..")

In [2]:
from os import listdir
from os.path import join, abspath
from MISalign.model.mis_file import MisFile,save_mis

# Basic Setup

In [3]:
###
# Enter setup information
###
folder_path=r"..\example\data\set_a" # folder with images
misfile_name="mymis_calibrated.mis" # name for save file
#
calibration_filepath=abspath(r"..\example\data\set_a\a_5x.miscal") # filepath to calibration file `.miscal`
    # Important: If you do not have a calibration file then uncomment the next line
# calibration_filepath=None
#
file_ending="jpg" # file extension
file_contains="" # file names must contain - i.e. "sample1-5x", "sample2", "10x", etc.
file_notcontains="" # file names must not contain - i.e. "calibration"
#TODO Change not contains to a list of strings to check against.
###

In [ ]:
mis_filepath=abspath(join(folder_path,misfile_name))
print(f"Save file filepath: {mis_filepath}")
file_names=[x for x in listdir(folder_path) if x.split(".")[-1]==file_ending]
if file_contains != "":
    file_names=[x for x in file_names if file_contains in x]
if file_notcontains != "":
    file_names=[x for x in file_names if file_notcontains not in x]
file_paths=[abspath(join(folder_path,x)) for x in file_names]
print("\n  ".join(["Image files and filepaths:"]+[f"{fn}: {fp}" for fn,fp in zip(file_names,file_paths)]))
if calibration_filepath is not None:
    print(f"Calibration file filepath: {calibration_filepath}")

In [ ]:
mis_project=MisFile(image_fps=file_paths,calibration_fp=calibration_filepath)
print(mis_project)

In [6]:
save_mis(mis_filepath,mis_project)

# Advanced Setups

## Image Sets

In [41]:
###
# Enter setup information
###
folder_path=r"..\example\data\set_a" # folder with images
misfile_names=["mymis1.mis","mymis2.mis"] # name for save file
#
file_ending="jpg" # file extension
file_contains="" # file names must contain - i.e. "sample1-5x", "sample2", "10x", etc.
file_notcontains="" # file names must not contain - i.e. "calibration"
###

In [ ]:
mis_filepaths=[abspath(join(folder_path,x)) for x in misfile_names]
mis_dict={mn:mp for mn,mp in zip(misfile_names,mis_filepaths)}
print("\n  ".join(["Save files and filepaths: "]+[f"{mn}: {mp}" for mn,mp in zip(misfile_names,mis_filepaths)]))
file_names=[x for x in listdir(folder_path) if x.split(".")[-1]==file_ending]
if file_contains != "":
    file_names=[x for x in file_names if file_contains in x]
if file_notcontains != "":
    file_names=[x for x in file_names if file_notcontains not in x]
file_paths=[abspath(join(folder_path,x)) for x in file_names]
print("\n  ".join(["Image index, file names, and filepaths:"]+[f"{i} - {fn}: {fp}" for i,(fn,fp) in enumerate(zip(file_names,file_paths))]))

In [ ]:
# user selected image sets using start and end index
image_sets={
    "mymis1.mis":(0,4),
    "mymis2.mis":(5,9),
}
for key,value in image_sets.items():
    print(key,file_names[value[0]:value[1]+1])

In [ ]:
for key,value in image_sets.items():
    mis_fp=mis_dict[key]
    print(mis_fp,"\n  ",file_names[value[0]:value[1]+1])
    mis_project=MisFile(image_fps=file_paths[value[0]:value[1]+1])
    save_mis(mis_fp,mis_project)

## SEM Trim & Convert
For trimming footer of SEM images and converting into uint8/RGB which is presently needed for further processing

In [ ]:
#TODO Remove need for RGB/uint8 conversion

In [1]:
from sys import path as syspath
syspath.append("..")

In [2]:
from MISalign.model.mis_file import load_mis
from PIL import Image as PILImage
import numpy as np
from shutil import copy2
from os.path import split, join
from os import mkdir

In [ ]:
mis_fp=r".mis"
mis_project=load_mis(mis_fp)
mis_image_paths=mis_project.get_image_paths()
mis_image_paths

In [ ]:
split_mis_fp=split(mis_fp)
original_image_folder=join(split_mis_fp[0],"original_images")
mkdir(original_image_folder)
for img_name, img_fp in mis_image_paths.items():
    split_img_fp=split(img_fp)
    copy_img_fp=join(original_image_folder,split_img_fp[1])
    print(img_fp," > ",copy_img_fp)
    copy2(img_fp,copy_img_fp)

In [7]:
pixels_to_trim=248
for img_name, img_fp in mis_image_paths.items():
    img_arr=np.asarray(PILImage.open(img_fp))
    if img_arr.dtype=="uint16":
        img_arr=(img_arr//256).astype(np.uint8)
    img_arr_trimmed=img_arr[:-248]
    img_corrected=PILImage.fromarray(img_arr_trimmed).convert('RGB')
    img_corrected.save(img_fp)